In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
plt.style.use("ggplot")

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

# Preprocessing

In [11]:
data_raw = pd.read_csv("monthly.csv")
y_raw = pd.read_csv("y.csv")
y_raw.head()

,date,TICKER,Y
0,19650129,NCM,1
1,19650129,ACD,0
2,19650129,ABP,1
3,19650129,T,1
4,19650129,AME,1


In [35]:
data_raw.head()

,PERMNO,date,SHRCD,TICKER,COMNAM,PRC,VOL,RET,RETX,dateDT,year
0,10057,1965-01-29,11.0,NCM,NATIONAL ACME CO,71.75,110.0,0.039855,0.039855,1965-01-29,1965
1,10057,1965-02-26,11.0,NCM,NATIONAL ACME CO,71.25,82.0,0.001742,-0.006969,1965-02-26,1965
2,10057,1965-03-31,11.0,NCM,NATIONAL ACME CO,76.00,243.0,0.066667,0.066667,1965-03-31,1965
3,10057,1965-04-30,11.0,NCM,NATIONAL ACME CO,39.50,152.0,0.039474,0.039474,1965-04-30,1965
4,10057,1965-05-28,11.0,NCM,NATIONAL ACME CO,37.00,256.0,-0.054430,-0.063291,1965-05-28,1965


In [32]:
if isinstance(data_raw['date'])
data_raw["date"] = data_raw["date"].apply(lambda x: str(x)[:4]+'-'+str(x)[4:6]+'-'+str(x)[6:])
data_raw["dateDT"] = pd.to_datetime(data_raw["date"])
data_raw["year"] = [_.year for _ in data_raw["dateDT"]]

In [34]:
data_raw.sort()

,PERMNO,date,SHRCD,TICKER,COMNAM,PRC,VOL,RET,RETX,dateDT
year,,,,,,,,,,
1965,2794,2794,2783,2783,2783,2774,2779,2774,2774,2794
1966,2837,2837,2832,2832,2832,2819,2823,2819,2819,2837
1967,2856,2856,2849,2842,2849,2833,2838,2840,2840,2856
1968,2923,2923,2907,2885,2907,2879,2883,2901,2901,2923
1969,3048,3048,3032,3008,3032,3002,3006,3026,3026,3048
1970,3175,3175,3163,3139,3163,3137,3139,3161,3161,3175
1971,3380,3380,3365,3341,3365,3336,3340,3360,3360,3380
1972,3604,3604,3521,3471,3521,3475,3405,3519,3519,3604
1973,4414,4414,4406,4331,4406,4354,3486,4383,4383,4414


# Descriptive Analysis

In [18]:
170385/(105177+41408+170385)

0.5375429851405495

In [19]:
(105177+41408+170385)

316970

In [41]:
data_raw.shape[0]*0.6

165667.19999999998

In [42]:
data_raw.iloc[165667, :]

PERMNO                  57817
date               1979-09-28
SHRCD                      11
TICKER                   NOBE
COMNAM          NORDSTROM INC
PRC                   -21.125
VOL                       NaN
RET                 -0.011696
RETX                -0.011696
dateDT    1979-09-28 00:00:00
year                     1979
Name: 165667, dtype: object

In [ ]:
data_raw[]